## 1)Hierarchical Indexing

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [3]:
data

a  1    0.026286
   2   -0.946957
   3   -0.579594
b  1    1.346484
   3    0.390191
c  1   -2.148270
   2   -0.511556
d  2   -0.316734
   3   -0.242866
dtype: float64

In [4]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [5]:
data.loc[['b','d']]

b  1    1.346484
   3    0.390191
d  2   -0.316734
   3   -0.242866
dtype: float64

In [6]:
data.loc[:, 2]

a   -0.946957
c   -0.511556
d   -0.316734
dtype: float64

In [7]:
data.unstack()                                        # data.unstack() :rearranging into DataFrame

,1,2,3
a,0.026286,-0.946957,-0.579594
b,1.346484,NaN,0.390191
c,-2.148270,-0.511556,NaN
d,NaN,-0.316734,-0.242866


In [8]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [9]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [10]:
frame.loc['a', ('Ohio', 'Green')]

key2
1    0
2    3
Name: (Ohio, Green), dtype: int32

In [11]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
names=['state', 'color'])                                                   # pd.Multiindex.from_arrays()

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## Reordering and Sorting Levels

In [12]:
frame.swaplevel('key1', 'key2')                          # df.swaplevel('','') : the data is unaltered

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [13]:
frame.sort_index(level=1)                                # df.sort_index(level=...)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [14]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## Summary Statistics by Level

In [15]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [16]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## Indexing with a DataFrame’s columns

In [17]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})

In [18]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [19]:
frame2 = frame.set_index(['c','d'])                # df.set_index(..., drop = False)
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [20]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [21]:
frame2.reset_index()                                    # df.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 2)Combining and Merging Datasets

## Database-Style DataFrame Joins

In [22]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

In [23]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

In [24]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [25]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [26]:
pd.merge(df1, df2, on='key')         # pd.merge(df1, df2, on='...', how='...')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


column names are different

In [27]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

In [28]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})

In [29]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')         # pd.merge(df1, df2, left_on='...', right_on='...')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [30]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


Many-to-many merges

In [31]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})

In [32]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})

In [33]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [34]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [35]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [36]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})

In [37]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

In [38]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [39]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right')) # suffixes=('...','...')

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


#### Merge function arguments: Python for Data Analysis, page 232

## Merging on Index

In [40]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})

In [41]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [42]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [43]:
right1

,group_val
a,3.5
b,7.0


In [44]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [45]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})

In [46]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [47]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [48]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [49]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [50]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                               index=['b', 'c', 'd', 'e'],
                               columns=['Missouri', 'Alabama'])

In [51]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [52]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [53]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame has a convenient join instance for merging by index.

In [54]:
left2.join(right2, how='outer')                        # df1.join(df2, how='', on='')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [55]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [56]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

In [57]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [58]:
left2.join([right2,another], how='outer', sort=True)

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


## Concatenating Along an Axis

In [59]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [60]:
pd.concat([s1,s2,s3])                           # pd.concat(join = '...', axis=, keys=, ignore_index=)

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [61]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [62]:
s4 = pd.concat([s1, s3])

In [63]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [64]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [65]:
s1 = s1.reindex(['a', 'c', 'b', 'e'])        # вместо join_index(не действителен)

In [66]:
s4 = s4.reindex(['a', 'c', 'b', 'e'])

In [67]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [68]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0.0
b,1.0,1.0
c,NaN,NaN
e,NaN,NaN


In [69]:
pd.concat([s1, s4], axis=1, keys=[3, 4])

,3,4
a,0.0,0.0
b,1.0,1.0
c,NaN,NaN
e,NaN,NaN


In [70]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [71]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

In [72]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [73]:
df2

,three,four
a,5,6
c,7,8


In [74]:
pd.concat([df1,df2], axis=1, keys=['level_1', 'level_2'])

level_1     level_2     
      one two   three four
a       0   1     5.0  6.0
b       2   3     NaN  NaN
c       4   5     7.0  8.0

In [75]:
#passing a dict
pd.concat({'level1': df1, 'level2': df2}, axis=1) 

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [76]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],          # names = []  
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

A last consideration concerns DataFrames in which the row index does not contain
any relevant data:

In [77]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [78]:
df1

,a,b,c,d
0,0.853843,0.248873,-0.371022,0.869836
1,0.544415,0.857682,0.213905,1.216958
2,-0.531517,2.791808,0.128997,-0.716548


In [79]:
df2

,b,d,a
0,-0.205009,-0.223641,0.586108
1,0.405487,0.733401,-0.378542


In [80]:
pd.concat([df1,df2], ignore_index=True)

,a,b,c,d
0,0.853843,0.248873,-0.371022,0.869836
1,0.544415,0.857682,0.213905,1.216958
2,-0.531517,2.791808,0.128997,-0.716548
3,0.586108,-0.205009,NaN,-0.223641
4,-0.378542,0.405487,NaN,0.733401


#### Concat function arguments: Python for Data Analysis, page 240

## Combining Data with Overlap

In [81]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])

In [82]:
b[-1] = np.nan

In [83]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [84]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [85]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [86]:
b[:-2]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

In [87]:
a[2:]

d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [88]:
b[:-2].combine_first(a[2:])               # ser1.combine_first(ser2):

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [89]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [90]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [91]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [92]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 3)Reshaping and Pivoting

## Reshaping with Hierarchical Indexing

In [98]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                                     name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [99]:
result = data.stack()                           # df.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [100]:
result.unstack()                               # df.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [101]:
#passing different level of multiindex (name or number)
result.unstack(0) 

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [103]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [104]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [105]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [106]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [107]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## Pivoting “Long” to “Wide” Format

In [121]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [122]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')

columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

data = data.reindex(columns=columns)

data.index = periods.to_timestamp('D', 'end')

data.head()

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2


In [123]:
data.stack()

date                           item   
1959-03-31 23:59:59.999999999  realgdp     2710.349
                               infl           0.000
                               unemp          5.800
1959-06-30 23:59:59.999999999  realgdp     2778.801
                               infl           2.340
                                            ...    
2009-06-30 23:59:59.999999999  infl           3.370
                               unemp          9.200
2009-09-30 23:59:59.999999999  realgdp    12990.341
                               infl           3.560
                               unemp          9.600
Length: 609, dtype: float64

In [124]:
data.stack().reset_index()

,date,item,0
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,unemp,9.200
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


In [125]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata.head()

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340


In [126]:
pivoted1 = ldata.pivot('date', 'item', 'value')          # df.pivot(col_as_rows,col_as_cols, col_as_fill)
pivoted1

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [127]:
ldata['value2'] = np.random.randn(len(ldata))
ldata.head(10)

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.848862
1,1959-03-31 23:59:59.999999999,infl,0.000,0.901876
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.841575
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,1.536097
4,1959-06-30 23:59:59.999999999,infl,2.340,1.187152
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.007845
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-0.276347
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.825677
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.447267
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-0.364194


In [128]:
pivoted2 = ldata.pivot('date', 'item')
pivoted2

value                     value2            \
item                           infl    realgdp unemp      infl   realgdp   
date                                                                       
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8  0.901876  0.848862   
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1  1.187152  1.536097   
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3 -0.825677 -0.276347   
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6 -0.745097 -0.364194   
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2 -0.772641 -1.060756   
...                             ...        ...   ...       ...       ...   
2008-09-30 23:59:59.999999999 -3.16  13324.600   6.0  0.425272 -2.196958   
2008-12-31 23:59:59.999999999 -8.79  13141.920   6.9 -0.037001 -0.409335   
2009-03-31 23:59:59.999999999  0.94  12925.410   8.1  0.176166 -0.164666   
2009-06-30 23:59:59.999999999  3.37  12901.504   9.2  0.973986 -0.670724   
2009-09-30 23:59:59.999999999  3.56  12990.341   9.6 -1.887713 -0.392167   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.841575  
1959-06-30 23:59:59.999999999  0.007845  
1959-09-30 23:59:59.999999999  0.447267  
1959-12-31 23:59:59.999999999 -1.155482  
1960-03-31 23:59:59.999999999  0.921698  
...                                 ...  
2008-09-30 23:59:59.999999999  0.111347  
2008-12-31 23:59:59.999999999 -0.522705  
2009-03-31 23:59:59.999999999  1.038991  
2009-06-30 23:59:59.999999999  2.239750  
2009-09-30 23:59:59.999999999 -0.018997  

[203 rows x 6 columns]

NOTE that pivot is equivalent to creating a hierarchical index using set_index followed
by a call to unstack

In [129]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked

value                     value2            \
item                           infl    realgdp unemp      infl   realgdp   
date                                                                       
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8  0.901876  0.848862   
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1  1.187152  1.536097   
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3 -0.825677 -0.276347   
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6 -0.745097 -0.364194   
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2 -0.772641 -1.060756   
...                             ...        ...   ...       ...       ...   
2008-09-30 23:59:59.999999999 -3.16  13324.600   6.0  0.425272 -2.196958   
2008-12-31 23:59:59.999999999 -8.79  13141.920   6.9 -0.037001 -0.409335   
2009-03-31 23:59:59.999999999  0.94  12925.410   8.1  0.176166 -0.164666   
2009-06-30 23:59:59.999999999  3.37  12901.504   9.2  0.973986 -0.670724   
2009-09-30 23:59:59.999999999  3.56  12990.341   9.6 -1.887713 -0.392167   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.841575  
1959-06-30 23:59:59.999999999  0.007845  
1959-09-30 23:59:59.999999999  0.447267  
1959-12-31 23:59:59.999999999 -1.155482  
1960-03-31 23:59:59.999999999  0.921698  
...                                 ...  
2008-09-30 23:59:59.999999999  0.111347  
2008-12-31 23:59:59.999999999 -0.522705  
2009-03-31 23:59:59.999999999  1.038991  
2009-06-30 23:59:59.999999999  2.239750  
2009-09-30 23:59:59.999999999 -0.018997  

[203 rows x 6 columns]

## Pivoting “Wide” to “Long” Format

In [130]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [131]:
melted = pd.melt(df, ['key'])                        # pd.melt() : inverse to pivot()
melted 

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [132]:
# original layout
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [133]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [134]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [135]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [136]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
